In [16]:
import torch
if torch.cuda.is_available():
    import torch.cuda as t
else:
    import torch as t
    
from   torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [2]:
import json, os
from pathlib import Path

In [3]:
class SimpleCNN(nn.Module):
    def __init__(self, n_classes=2, kernels=4, layers=1, max_pool_size=0):
        super(SimpleCNN, self).__init__()
        self.conv = nn.Sequential()
        in_channels = 1
        out_channels = kernels
        for n in range(layers):
            self.conv.add_module('conv'+str(n), nn.Conv2d(in_channels, out_channels, 3))
            in_channels = out_channels
            out_channels += 2
        self.maxpool = nn.MaxPool2d(max_pool_size)
        self.dropout = nn.Dropout()
        self.out_dim = (out_channels -2) * ((28-layers * 2 )**2)
        
        self.fc1 = nn.Linear(self.out_dim,100)
        self.fc2 = nn.Linear(100, n_classes)
            
        
    def forward(self, x):
        x = self.conv(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = x.view(-1,self.out_dim)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.softmax(x, dim=1)
        return x

In [4]:
# s = nn.Sequential()
# net.conv.add_module('conv2', nconv)

In [5]:

# for data in testloader:
#     images, labels = data
net = SimpleCNN(n_classes=10,layers=2,kernels=10)
net.conv.children

<bound method Module.children of Sequential(
  (conv0): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1))
  (conv1): Conv2d(10, 12, kernel_size=(3, 3), stride=(1, 1))
)>

In [6]:
config = json.load(open(os.path.expanduser("~/.thesis.conf")))
root = Path(config['datasets']) / Path("MNIST/") 
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
mnist_set = torchvision.datasets.MNIST(str(root),train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(mnist_set, batch_size=25,
                                          shuffle=True, num_workers=2)
testset = torchvision.datasets.MNIST(str(root), train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

dataiter = iter(trainloader)
images, label = dataiter.next()

In [7]:
type(mnist_set)

torchvision.datasets.mnist.MNIST

In [8]:
net = SimpleCNN(10,layers=3)#.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
# loss_

In [9]:

for epoch in range(1):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = Variable(inputs), Variable(labels)
        
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.data[0]
        if i % 200 == 199:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')

[1,   200] loss: 2.302
[1,   400] loss: 2.302
[1,   600] loss: 2.301
[1,   800] loss: 2.299
[1,  1000] loss: 2.297
[1,  1200] loss: 2.291
[1,  1400] loss: 2.259
[1,  1600] loss: 1.982
[1,  1800] loss: 1.701
[1,  2000] loss: 1.647
[1,  2200] loss: 1.633
[1,  2400] loss: 1.619
Finished Training


In [10]:
correct = 0
total = 0
for data in testloader:
    images, labels = data
    imgs = Variable(images)#.cuda()
    outputs = net(imgs)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
    
print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 86 %


In [11]:
from torch.utils.trainer import Trainer
from torch.utils.trainer.plugins import Logger, LossMonitor, TimeMonitor, ProgressMonitor, Logger

In [12]:
net = SimpleCNN(n_classes=10,layers=2,kernels=10)#.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

trainer = Trainer(net, criterion, optimizer, testloader)
trainer.register_plugin(LossMonitor())
trainer.register_plugin(TimeMonitor())
trainer.register_plugin(ProgressMonitor())
trainer.register_plugin(Logger(['loss', 'time'],interval=[(500, 'iteration'), (1, 'epoch')]))

In [13]:
trainer.run(epochs=3)

loss: 2.3276  (2.2334)	time: 11ms  (11ms)
loss: 1.7972  (1.8273)	time: 11ms  (12ms)
loss: 1.7502  (1.7211)	time: 12ms  (12ms)
loss: 1.7988  (1.5839)	time: 16ms  (14ms)
loss: 1.9491  (1.7229)	time: 12ms  (14ms)
################################################################################
Epoch summary:
loss: 1.8918	time: 12ms
################################################################################
loss: 1.6161  (1.6947)	time: 14ms  (14ms)
loss: 1.7106  (1.6205)	time: 15ms  (15ms)
loss: 1.7083  (1.5643)	time: 13ms  (13ms)
loss: 1.5289  (1.5143)	time: 13ms  (14ms)
loss: 1.8535  (1.6383)	time: 14ms  (15ms)
################################################################################
Epoch summary:
loss: 1.6332	time: 14ms
################################################################################
loss: 1.4613  (1.5852)	time: 67ms  (30ms)
loss: 1.4613  (1.5646)	time: 15ms  (14ms)
loss: 1.6877  (1.6305)	time: 14ms  (14ms)
loss: 1.6462  (1.5964)	time: 13ms  (13ms)
loss: 1.51

In [43]:
trainer.stats

{'loss': {'epoch_mean': 1.5414614585240682,
  'epoch_stats': (0, 0),
  'last': 1.561108946800232,
  'log_epoch_fields': ['{epoch_mean:.4f}'],
  'log_format': ':.4f',
  'log_iter_fields': ['{last:.4f}', ' ({running_avg:.4f})'],
  'log_unit': '',
  'running_avg': 1.52145434676333},
 'progress': {'epoch_size': 2500,
  'log_iter_fields': ['{samples_used}/{epoch_size}', '({percent:.2f}%)'],
  'percent': 0,
  'samples_used': 0},
 'time': {'epoch_mean': 9.27558266321818,
  'epoch_stats': (0, 0),
  'last': 5.194902420043945,
  'log_epoch_fields': ['{epoch_mean:.0f}ms'],
  'log_format': ':.0f',
  'log_iter_fields': ['{last:.0f}ms', ' ({running_avg:.0f}ms)'],
  'log_unit': 'ms',
  'running_avg': 5.565947944499839}}